In [14]:
import pandas as pd
import numpy as np

In [15]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [16]:
from IPython.display import HTML

In [ ]:
from models import createModel
from animate import AnimateFeature

In [17]:
# import model structure from models.py file
model = createModel()

In [19]:
# load model weights
model_string = f"models/fifth_model/weights_epochs20"
model.load_weights(model_string)

In [20]:
x_val = np.load("./cleaned_data/x_val.npy")
y_val = np.load("./cleaned_data/y_val.npy")
x_test = np.load("./cleaned_data/x_test.npy")
y_test = np.load("./cleaned_data/y_test.npy")
train_mu = np.load("./cleaned_data/train_mu.npy")
train_std = np.load("./cleaned_data/train_std.npy")


In [21]:
preds = model.predict(x_val[:,:,4:])

1317/1317 [==============================] - 2s 2ms/step


In [25]:
# undo normalization for the input, so we can pass it into play visualizer
unadj_x_val = x_val
unadj_x_val[:,:,5:] = (x_val[:,:,5:]*train_std[5:])+train_mu[5:]
unadj_x_val[:,:,6] += (53.3/2)
        

In [37]:
preds_repeat = np.tile(preds, (1,23)).reshape(-1,23,3)
val_with_preds = np.concatenate((unadj_x_val, preds_repeat), axis=2)


In [39]:
col_names = ['gameId', 'playId', 'frameId', 'nflId', 'team_indicator', 'adj_x', 'adj_y', 's', 'a', 'adj_o', 'adj_dir', 'no_sack_prob', 'sack_prob', 'pred_time']

val_with_preds_df = pd.DataFrame(val_with_preds.reshape(-1, len(col_names)), columns=col_names)
val_with_preds_df.gameId = val_with_preds_df.gameId.astype('int64')


In [78]:
val_with_preds_df['true_sack'] = np.tile(y_val[:,1], 23)
#np.repeat(y_val[:,1], 23)

In [79]:
val_with_preds_df.head()

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
0,2021101704,521.0,6.0,38544.0,1.0,86.66,35.35,0.00,7.000000e-02,170.22,156.66,0.5461,0.4539,-0.730734,0.0
1,2021101704,521.0,6.0,38553.0,2.0,88.47,33.00,0.00,2.220446e-16,327.67,261.74,0.5461,0.4539,-0.730734,0.0
2,2021101704,521.0,6.0,40171.0,1.0,86.74,32.14,0.02,3.700000e-01,174.35,153.68,0.5461,0.4539,-0.730734,0.0
3,2021101704,521.0,6.0,41619.0,2.0,87.92,29.86,0.01,1.200000e-01,338.62,339.74,0.5461,0.4539,-0.730734,0.0
4,2021101704,521.0,6.0,42444.0,1.0,86.47,25.06,0.01,1.900000e-01,145.39,156.39,0.5461,0.4539,-0.730734,0.0


In [80]:
true_sack_df = val_with_preds_df.query("true_sack == 1")
true_sack_df.gameId = true_sack_df.gameId.astype('int64')

/var/folders/59/2qy07m9n56j1hjrbkcj93rhc0000gn/T/ipykernel_10841/1917819778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_sack_df.gameId = true_sack_df.gameId.astype('int64')


In [81]:
true_sack_df.drop_duplicates(['gameId', 'playId'])

,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
186,2021101704,521.0,14.0,40171.0,1.0,87.20,32.46,1.13,5.000000e-02,165.88,145.97,0.455943,0.544057,0.279656,1.0
1435,2021101704,566.0,33.0,44888.0,1.0,69.15,31.30,7.18,8.500000e-01,72.93,30.91,0.950853,0.049147,0.821917,1.0
2324,2021101704,617.0,20.0,38553.0,2.0,79.88,34.77,1.79,2.810000e+00,79.67,158.85,0.619058,0.380942,0.394696,1.0
3237,2021101704,680.0,22.0,53434.0,2.0,91.47,9.46,4.89,2.550000e+00,13.13,313.61,0.735789,0.264211,-0.099236,1.0
4300,2021101704,746.0,6.0,0.0,3.0,48.01,23.70,0.00,2.220446e-16,0.00,0.00,0.761281,0.238719,0.190733,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962919,2021102407,2592.0,10.0,38592.0,1.0,32.69,8.57,0.26,2.080000e+00,148.93,335.00,0.664722,0.335278,-0.213167,1.0
964229,2021102407,2614.0,16.0,37266.0,2.0,40.38,27.45,2.32,8.000000e-01,6.91,165.41,0.878815,0.121185,1.187281,1.0
964781,2021102407,2869.0,6.0,37266.0,2.0,64.05,23.32,0.19,6.600000e-01,325.46,49.34,0.355578,0.644422,-0.863158,1.0
966575,2021102407,2890.0,30.0,37266.0,2.0,60.54,26.12,0.53,1.120000e+00,263.47,215.87,0.880760,0.119240,-0.472816,1.0


In [82]:
# test_game_id = 2021101704
# test_play_id = 746.0
# good example of probability increasing as defender comes near

test_game_id = 2021102407
test_play_id = 2614.0

In [83]:

play_df = val_with_preds_df.query("gameId == @test_game_id and playId == @test_play_id")
print(f"play_df shape = {play_df.shape}")

play_df.head()

play_df shape = (782, 15)


,gameId,playId,frameId,nflId,team_indicator,adj_x,adj_y,s,a,adj_o,adj_dir,no_sack_prob,sack_prob,pred_time,true_sack
963999,2021102407,2614.0,6.0,37266.0,2.0,38.81,27.00,0.11,0.95,319.50,194.42,0.813024,0.186976,0.927457,0.0
964000,2021102407,2614.0,6.0,38592.0,1.0,32.16,8.02,0.18,0.42,70.35,339.14,0.813024,0.186976,0.927457,0.0
964001,2021102407,2614.0,6.0,39950.0,2.0,39.85,29.65,0.93,2.10,16.01,151.07,0.813024,0.186976,0.927457,0.0
964002,2021102407,2614.0,6.0,42391.0,1.0,32.05,22.27,0.15,0.58,160.29,315.43,0.813024,0.186976,0.927457,0.0
964003,2021102407,2614.0,6.0,43358.0,1.0,37.52,21.59,0.10,0.49,177.97,195.82,0.813024,0.186976,0.927457,0.0


In [85]:
animated_play = AnimateFeature(play_df)
HTML(animated_play.ani.to_jshtml())

In [48]:
# from importlib import reload

# def importOrReload(module_name, *names):
#     import sys

#     if module_name in sys.modules:
#         reload(sys.modules[module_name])
#     else:
#         __import__(module_name, fromlist=names)

#     for name in names:
#         globals()[name] = getattr(sys.modules[module_name], name)
        
# importOrReload("temp", "AnimatePlay")
